In [2]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Convolution2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras import backend as K
from keras.utils import np_utils
import pandas
import sys
import os
import numpy as np

Using TensorFlow backend.
C:\Users\Bloris\Anaconda3\envs\bloris\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Bloris\Anaconda3\envs\bloris\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Bloris\Anaconda3\envs\bloris\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Bloris\Anaconda3\envs\bloris\lib

In [7]:

def file_read(data_path):
    x,y=[],[]
    print('Loading data... ', end='')
    for root, subFolder, files in os.walk(data_path):
        for item in files:
            if item.endswith(".txt"):
                fileNamePath = str(os.path.join(root,item))
                x1,x2=[],[]
                f = open(fileNamePath, 'r')
                
                tmp=list(map(int, f.readline()[:-1]))
                y.append(tmp)

                for i in range(3):
                    tmp=list(map(int, f.readline()[:-1].split(' ')[:-1]))
                    x1.append([tmp])
    
                f.readline()
                f.readline()
    
                for i in range(3):
                    tmp=list(map(int, f.readline()[:-1].split(' ')[:-1]))
                    x2.append([tmp])
    
                x1 = list(np.array(x1).T)
                x2 = list(np.array(x2).T)
    
                x.append([x1,x2])
    f.close()
    print('%d data loaded.'%len(x))
    return x,y

In [12]:
XH,YH=file_read('hnn/train')
XH,YH=np.array(XH),np.array(YH)

Loading data... 443 data loaded.


In [13]:
XH = XH.reshape(len(x),2,256,3)
XH.shape

print(x.shape)
print(x.shape)
print(y.shape)
XH = XH.astype('float32')
XH /= 255
YH = np_utils.to_categorical(YH)

idx=np.arange(443)
np.random.shuffle(idx)

(443, 2, 256, 3)
(443, 2, 256, 3)
(443, 2, 2)


In [19]:
from keras import optimizers

Nadam = optimizers.Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.003)

cnn = Sequential()
cnn.add(Convolution2D(64, 2, 3,border_mode="valid",activation="elu",input_shape=(2, 256, 3)))
cnn.add(Dropout(0.5))

cnn.add(Convolution2D(32, 1, 1, border_mode="valid", activation="elu"))
cnn.add(Dropout(0.5))
cnn.add(Flatten())

cnn.add(Dense(8, activation="elu"))
cnn.add(Dropout(0.5))

cnn.add(Dense(2, activation="softmax"))

cnn.summary()

cnn.compile(loss='categorical_crossentropy', optimizer=Nadam, metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 1, 254, 64)        1216      
_________________________________________________________________
dropout_7 (Dropout)          (None, 1, 254, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 1, 254, 32)        2080      
_________________________________________________________________
dropout_8 (Dropout)          (None, 1, 254, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 8128)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 65032     
_________________________________________________________________
dropout_9 (Dropout)          (None, 8)                 0         
__________

C:\Users\Bloris\Anaconda3\envs\bloris\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 3), activation="elu", input_shape=(2, 256, 3..., padding="valid")`
  
C:\Users\Bloris\Anaconda3\envs\bloris\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (1, 1), activation="elu", padding="valid")`
  if __name__ == '__main__':


In [20]:
cnn.fit(XH, YH, batch_size=4, nb_epoch=50, verbose=1)

Epoch 1/50
443/443 [==============================] - 1s 1ms/step - loss: 2.4644 - acc: 0.4989
Epoch 2/50
443/443 [==============================] - 0s 691us/step - loss: 1.2864 - acc: 0.5440
Epoch 3/50
443/443 [==============================] - 0s 669us/step - loss: 1.0329 - acc: 0.5824
Epoch 4/50
443/443 [==============================] - 0s 680us/step - loss: 0.9606 - acc: 0.5688
Epoch 5/50
443/443 [==============================] - 0s 669us/step - loss: 0.9657 - acc: 0.5372
Epoch 6/50
443/443 [==============================] - 0s 682us/step - loss: 0.9272 - acc: 0.5711
Epoch 7/50
443/443 [==============================] - 0s 664us/step - loss: 0.7970 - acc: 0.5869
Epoch 8/50
443/443 [==============================] - 0s 653us/step - loss: 0.8102 - acc: 0.6208
Epoch 9/50
443/443 [==============================] - 0s 673us/step - loss: 0.7653 - acc: 0.6117
Epoch 10/50
443/443 [==============================] - 0s 667us/step - loss: 0.7729 - acc: 0.6433
Epoch 11/50
443/443 [==========

In [10]:
from keras.models import *

from keras.utils import *

cnn.save('weights_hnn_fit_aug.h5')